In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

import jax
from jax import numpy as jnp
from jax.config import config
from flax.training import checkpoints
from flax import linen as nn

import nest_asyncio
nest_asyncio.apply()

from scipy.interpolate import interp1d

# Constants
from scipy.constants import Boltzmann, Avogadro
kb = Boltzmann # [J/K] Boltzman's constant
Na = Avogadro  # [mol-1] Avogadro's Number
R = Na * kb    # [J mol-1 K-1] Ideal gas constant

sys.path.append("../../")
from python_helpers.feanneos import HelmholtzModel
from python_helpers.feanneos import helper_solver_funs, helper_jitted_funs
from python_helpers.transport_properties import TransportModel_PVT_Tinv
from python_helpers import helper_get_alpha
from python_helpers import linear_activation
from python_helpers.data_figures import data_phase_equilibria_mie_solid, data_phase_equilibria_mie_fluid, mie_data_si_units

PRECISSION = 'float64'
if PRECISSION == 'float64':
    config.update("jax_enable_x64", True)
    type_np = np.float64
    type_jax = jnp.float64
else:
    config.update("jax_enable_x32", True)
    type_np = np.float32
    type_jax = jnp.float32

np.seterr(all="ignore")

{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

In [2]:
folder_to_save = 'results'
os.makedirs(folder_to_save, exist_ok=True)

In [3]:
###########

filename = '../../computed_files/phase_equilibria_solid.xlsx'
excel_file = pd.ExcelFile(filename)

df_info = pd.read_excel(excel_file, sheet_name='info')

# critical point information interpolation
input_crit_interp = df_info['lambda_r'].to_numpy()
output_crit_interp = df_info[['rhocad_model', 'Tcad_model']].to_numpy()
crit_interp = interp1d(input_crit_interp, output_crit_interp.T, fill_value='extrapolate')

# triple point information interpolation
input_triple_interp = df_info['lambda_r'].to_numpy()
output_triple_interp = df_info[['rhovad_triple', 'rholad_triple', 'rhosad_triple',
                                'T_triple']].to_numpy()
triple_interp = interp1d(input_triple_interp, output_triple_interp.T, fill_value='extrapolate')

In [4]:
######################
# Loading FE-ANN EoS #
######################

ckpt_folder = '../../ann_models/feann_eos'

prefix_params = 'FE-ANN-EoS-params_'

###
Tscale = 'Tinv'
seed = 1
factor = 0.05
EPOCHS = 20000
traind_model_folder = f'models_{Tscale}_factor{factor:.2f}_seed{seed}'
ckpt_folder_model = os.path.join(ckpt_folder, traind_model_folder)
ckpt_Tinv = checkpoints.restore_checkpoint(ckpt_dir=ckpt_folder_model, target=None, prefix=prefix_params)
helmholtz_features = list(ckpt_Tinv['features'].values())
helmholtz_model = HelmholtzModel(features=helmholtz_features)
helmholtz_params = {'params': ckpt_Tinv['params']}
fun_dic_fluid = helper_jitted_funs(helmholtz_model, helmholtz_params)

#########################
# Loading FE-ANN(s) EoS #
#########################

ckpt_folder = '../../ann_models/feanns_eos'
prefix_params = 'FE-ANN-EoS-params_'
###
Tscale = 'Tinv'
seed = 17
factor = 0.01
EPOCHS = 50000
traind_model_folder = f'models_{Tscale}_factor{factor:.2f}_seed{seed}'
ckpt_folder_model = os.path.join(ckpt_folder, traind_model_folder)
ckpt_Tinv = checkpoints.restore_checkpoint(ckpt_dir=ckpt_folder_model, target=None, prefix=prefix_params, step=EPOCHS)
helmholtz_features = list(ckpt_Tinv['features'].values())
helmholtz_model = HelmholtzModel(features=helmholtz_features)
helmholtz_params = {'params': ckpt_Tinv['params']}

fun_dic_solid = helper_jitted_funs(helmholtz_model, helmholtz_params)

########################### 
# Self-diffusivity models #
###########################

folder_diff = '../../ann_models/selfdiff_models'
hidden_layers = 2
neurons = 30
prefix = 'rhodiff-rho-Tinv-penalty'
ckpt_dir = folder_diff
seed = 1
features = hidden_layers * [neurons]
activation = 'softplus'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=ckpt_dir, target=None, prefix=params_prefix)
params_rhodiff = {'params': state_restored['params']}
rhodiff_model = TransportModel_PVT_Tinv(features=features, output_activation=nn.softplus)

rhodiff_model_jit = jax.jit(lambda alpha, rhoad, Tad: rhodiff_model.apply(params_rhodiff, jnp.atleast_1d(alpha), jnp.atleast_1d(rhoad), jnp.atleast_1d(Tad)))
diff_fun = lambda alpha, rhoad, Tad: rhodiff_model_jit(alpha, rhoad, Tad)/rhoad

#########################
# Shear visocsity model #
#########################

folder_visc = '../../ann_models/visc_models'
hidden_layers = 2
neurons = 30
prefix = 'logvisc-rho-Tinv-penalty'
ckpt_dir = folder_visc
seed = 0
features = hidden_layers * [neurons]
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=ckpt_dir, target=None, prefix=params_prefix)
params_logvisc = {'params': state_restored['params']}
logvisc_model = TransportModel_PVT_Tinv(features=features)

logvisc_model_jit = jax.jit(lambda alpha, rhoad, Tad: logvisc_model.apply(params_logvisc, jnp.atleast_1d(alpha), jnp.atleast_1d(rhoad), jnp.atleast_1d(Tad)))
visc_fun = lambda alpha, rhoad, Tad: jnp.exp(logvisc_model_jit(alpha, rhoad, Tad))


##############################
# Thermal conductivity model #
##############################

folder_tcond = '../../ann_models/tcond_models'
hidden_layers = 3
neurons = 30
prefix = 'logtcond-rho-Tinv-penalty'
ckpt_dir = folder_tcond
seed = 1
features = hidden_layers * [neurons]
activation = 'linear'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=ckpt_dir, target=None, prefix=params_prefix)
params_logtcond = {'params': state_restored['params']}
logtcond_model = TransportModel_PVT_Tinv(features=features)

logtcond_model_jit = jax.jit(lambda alpha, rhoad, Tad: logtcond_model.apply(params_logtcond, jnp.atleast_1d(alpha), jnp.atleast_1d(rhoad), jnp.atleast_1d(Tad)))
tcond_fun = lambda alpha, rhoad, Tad: jnp.exp(logtcond_model_jit(alpha, rhoad, Tad))


In [5]:
# Reading data from NIST
fluid_name = r'Carbon Monoxide'
filename = 'carbon_monoxide.xlsx'
DataFile = pd.ExcelFile(filename)

non_vibrational_deg_freedom = 5. # 5 for linear molecules
def Cvid_by_R(T):
    # valid from  60 to 1500 K
    k1 = 29108.
    k2 = 8773.
    k3 = 3085.1
    k4 = 8455.3
    k5 = 1538.2
    
    Cp = k1 + k2 * ((k3/T) /np.sinh(k3/T))**2  # J / (kmol K) 
    Cp += k4 * ((k5/T) /np.cosh(k5/T))**2 # J / (kmol K)
    Cp /= 1000.  # J / (mol K)

    Cp_R = Cp / R # Adim
    Cv_R = Cp_R - 1. # Adim 
    return Cv_R

Mw = DataFile.parse('info')['Mw'][0] # g/mol


In [6]:
file_types = ['vle', 'vle_visc', 'vle_sle_sve']
dict_values = {}
# collecting the values

for file_type in file_types:
    file = f'optimized_mie_params_{file_type}.csv'
    df = pd.read_csv(file)
    df.sort_values('of', inplace=True, ignore_index=True)
    sigma, eps, lambda_r = df.loc[0, ['sigma_sol',	'epsilon_sol',	'lr_sol']].values

    initial_crit_point = crit_interp(lambda_r)
    initial_triple_point = triple_interp(lambda_r)

    data_df_fluid = data_phase_equilibria_mie_fluid(fun_dic_fluid, diff_fun, visc_fun, tcond_fun,
                                                    lambda_r=lambda_r,
                                                    initial_crit_point=initial_crit_point)

    data_df_solid = data_phase_equilibria_mie_solid(fun_dic_solid, diff_fun, visc_fun, tcond_fun,
                                                    lambda_r=lambda_r,
                                                    initial_triple_point=initial_triple_point, 
                                                    initial_crit_point=initial_crit_point)

    dict_values_mie_solid = mie_data_si_units(sigma, eps, Mw, data_df_solid, Cvid_by_R=Cvid_by_R, non_vibrational_deg_freedom=non_vibrational_deg_freedom, include_solid=True)
    dict_values_mie_fluid = mie_data_si_units(sigma, eps, Mw, data_df_fluid, Cvid_by_R=Cvid_by_R, non_vibrational_deg_freedom=non_vibrational_deg_freedom, include_solid=False)

    filename = f'mie_results_{file_type}_fluid.xlsx'
    file_to_save = os.path.join(folder_to_save, filename)
    writer = pd.ExcelWriter(file_to_save, engine='xlsxwriter')
    for key, df in dict_values_mie_fluid.items():
        df.to_excel(writer, sheet_name=key, index=False)
    writer.close()

    filename = f'mie_results_{file_type}_solid.xlsx'
    file_to_save = os.path.join(folder_to_save, filename)
    writer = pd.ExcelWriter(file_to_save, engine='xlsxwriter')
    for key, df in dict_values_mie_solid.items():
        df.to_excel(writer, sheet_name=key, index=False)
    writer.close()
    

2025-03-19 16:08:24.151483: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-03-19 16:08:24.613114: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-03-19 16:08:24.613374: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-03-19 16:08:24.721662: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-03-19 16:08:24.722696: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-03-19 16:08:26.812582: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-03-19 16:08:26.830537: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-03-19 16:08:26.838550: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
